# Fase 4 & 5: Modelagem e Avaliação

### Importações de bibliotecas e carregamento dos dados

In [2]:
import os, json
import numpy as np
import pandas as pd

# Modelos
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multiclass import OneVsRestClassifier

# Métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error

# Persistência
from joblib import dump


In [3]:
# Paths
PROC = "../data/processed"
MODELS = "../models"
REPORTS = "../reports"

os.makedirs(MODELS, exist_ok=True)
os.makedirs(REPORTS, exist_ok=True)

# Carregar datasets preparados na Fase 3
data         = pd.read_csv(f"{PROC}/imdb_clean.csv")
data_scaled  = pd.read_csv(f"{PROC}/imdb_clean_scaled.csv")
data_tfidf   = pd.read_csv(f"{PROC}/imdb_clean_tfidf.csv")
data_svd     = pd.read_csv(f"{PROC}/imdb_clean_svd.csv")
data_mean    = pd.read_csv(f"{PROC}/imdb_clean_mean.csv")

In [4]:
# Carregar features.json para saber exatamente os nomes de colunas
with open(f"{PROC}/features.json", "r") as f:
    FEATS = json.load(f)

print("Datasets carregados")

Datasets carregados


### Descobrir colunas de gênero e TF-IDF

In [5]:
def infer_label_cols_from_genre(df):
    # Função auxiliar --> quais colunas do DataFrame correspondem aos rótulos de gênero (one-hot encoded).
    # Encontra todos os gêneros únicos a partir da coluna 'Genre' original
    all_genres = set()
    for genre_list in df["Genre"].dropna():
        all_genres.update([s.strip() for s in genre_list.split(",")])

    # verificação de quais colunas do DataFrame correspondem a esses gêneros e são colunas binárias
    label_columns = []
    for col in df.columns:
        if col in all_genres:
            # checar se a coluna é binária
            is_binary = df[col].dropna().unique().tolist()
            if set(is_binary).issubset({0, 1}):
                label_columns.append(col)
    return sorted(label_columns)


In [6]:
# Identificar as colunas de gênero no dataset principal
label_cols = infer_label_cols_from_genre(data)
print(f"Colunas de gênero (multi-label) identificadas: {len(label_cols)}")
print(label_cols)

# Identificar as colunas TF-IDF
tfidf_cols = sorted(list(set(FEATS["data_tfidf"]) - set(FEATS["data"])))
print(f"\nTotal de colunas TF-IDF identificadas: {len(tfidf_cols)}")

Colunas de gênero (multi-label) identificadas: 21
['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']

Total de colunas TF-IDF identificadas: 500


### Regressão para Gross

In [7]:
# imports pro XGBoost e GridSearchCV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

print("--- Iniciando a Regressão para 'Gross' com Otimização (GridSearchCV) ---")

# TARGET E FEATURES
target_gross = "Gross"

Xg = data.select_dtypes(include=np.number).copy()
Xg = Xg.drop(columns=[target_gross], errors='ignore') # o alvo é removido

yg = data[target_gross].values

# treino e teste
Xg_train, Xg_test, yg_train, yg_test = train_test_split(Xg, yg, test_size=0.2, random_state=42)
print(f"Dados divididos em {len(Xg_train)} amostras de treino e {len(Xg_test)} de teste.")

--- Iniciando a Regressão para 'Gross' com Otimização (GridSearchCV) ---
Dados divididos em 664 amostras de treino e 166 de teste.


In [8]:
#  otimizaçaõ de hiperparametros
param_grid = {
    'n_estimators': [300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.7, 1.0]
}

# Instanciamos o modelo base
xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

# cv=3 --> 3 folds para cada combinação
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='r2',
    cv=3,
    verbose=1 # progresso do treinamento
)

In [9]:
print("\nIniciando a busca pelos melhores parâmetros (GridSearchCV)...")
grid_search.fit(Xg_train, yg_train)

print("\nBusca concluída.")
print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")

# melhor modelo já treinado:
best_xgb_gross = grid_search.best_estimator_


Iniciando a busca pelos melhores parâmetros (GridSearchCV)...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

Busca concluída.
Melhores parâmetros encontrados: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.7}


In [10]:
# avaliação do modelo otimizado
print("\nAvaliando o modelo otimizado...")
pred = best_xgb_gross.predict(Xg_test)

r2 = r2_score(yg_test, pred)
mae = mean_absolute_error(yg_test, pred)
rmse = np.sqrt(mean_squared_error(yg_test, pred))

print(f"\n[Resultado Gross Otimizado] R²={r2:.3f} | MAE={mae:,.2f} | RMSE={rmse:,.2f}")


Avaliando o modelo otimizado...

[Resultado Gross Otimizado] R²=0.616 | MAE=39,096,722.91 | RMSE=64,106,900.27


In [11]:
#  importancia das features
imp = pd.DataFrame({
    "feature": Xg.columns,
    "importance": best_xgb_gross.feature_importances_
}).sort_values("importance", ascending=False)

imp.to_csv(f"{REPORTS}/feature_importance_gross_optimized.csv", index=False)
print("\nTop 15 fatores para Gross (modelo otimizado):")
print(imp.head(15))

# salvar
dump(best_xgb_gross, f"{MODELS}/modelo_faturamento.pkl")
print(f"\nModelo otimizado 'modelo_faturamento.pkl' salvo na pasta '{MODELS}' ✔️")


Top 15 fatores para Gross (modelo otimizado):
        feature  importance
1     Adventure    0.310402
18  No_of_Votes    0.127449
14    Movie_Age    0.065129
7        Family    0.046099
0        Action    0.042734
12  IMDB_Rating    0.037704
21      Runtime    0.036583
22       Sci-Fi    0.032429
2     Animation    0.031974
8       Fantasy    0.031125
6         Drama    0.028520
13   Meta_score    0.026093
26      Western    0.024234
5         Crime    0.020366
4        Comedy    0.018717

Modelo otimizado 'modelo_faturamento.pkl' salvo na pasta '../models' ✔️


### Classificação multi-rótulo de Gênero via Overview

In [12]:
# Importando a Regressão Logística
from sklearn.linear_model import LogisticRegression

print("--- Rodando o melhor modelo para Classificação de Gênero (Regressão Logística) ---")

# target e features
Xt = data_tfidf[tfidf_cols].copy()
Yt = data[label_cols].copy()

# divisão treino/teste
Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(Xt, Yt, test_size=0.2, random_state=42)
print(f"Dados divididos em {len(Xt_train)} amostras de treino e {len(Xt_test)} de teste.")

--- Rodando o melhor modelo para Classificação de Gênero (Regressão Logística) ---
Dados divididos em 664 amostras de treino e 166 de teste.


In [13]:
# One-vs-Rest com Regressão Logística
print("\nTreinando o modelo...")
clf_genre_logreg = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, solver="liblinear", class_weight="balanced")
)
clf_genre_logreg.fit(Xt_train, Yt_train)
print("Treinamento concluído.")


Treinando o modelo...
Treinamento concluído.


In [14]:
# avaliacao
print("\nAvaliando o modelo...")
Yt_pred = clf_genre_logreg.predict(Xt_test)

f1_micro  = f1_score(Yt_test, Yt_pred, average='micro', zero_division=0)
f1_macro  = f1_score(Yt_test, Yt_pred, average='macro', zero_division=0)
prec_micro = precision_score(Yt_test, Yt_pred, average='micro', zero_division=0)
rec_micro  = recall_score(Yt_test, Yt_pred, average='micro', zero_division=0)

print(f"\n[Resultado Gêneros com Regressão Logística] F1-micro={f1_micro:.3f} | F1-macro={f1_macro:.3f} | Precision-micro={prec_micro:.3f} | Recall-micro={rec_micro:.3f}")

# salvar
dump(clf_genre_logreg, f"{MODELS}/modelo_genero.pkl")
print(f"\nMelhor modelo de gêneros ('modelo_genero.pkl') salvo na pasta '{MODELS}' ✔️")


Avaliando o modelo...

[Resultado Gêneros com Regressão Logística] F1-micro=0.319 | F1-macro=0.201 | Precision-micro=0.301 | Recall-micro=0.340

Melhor modelo de gêneros ('modelo_genero.pkl') salvo na pasta '../models' ✔️


### Regressão para IMDb Rating

In [15]:
# Importando as ferramentas necessárias
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

print("--- Iniciando a Regressão para 'IMDB_Rating' com Otimização ---")

# target e features
target_rating = "IMDB_Rating"

# para garantir que o XGBoost não receba colunas de texto.
Xr = data_scaled.select_dtypes(include=np.number).copy()
Xr = Xr.drop(columns=[target_rating], errors='ignore')

yr = data_scaled[target_rating].values

# treino e teste
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.2, random_state=42)
print(f"Dados divididos em {len(Xr_train)} amostras de treino e {len(Xr_test)} de teste.")

--- Iniciando a Regressão para 'IMDB_Rating' com Otimização ---
Dados divididos em 664 amostras de treino e 166 de teste.


In [16]:
# otimização
# "grid" de parâmetros para o XGBoost testar
param_grid_rating = {
    'n_estimators': [200, 400],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.7, 1.0]
}

# Instanciar o modelo base
xgb_rating_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

# Instanciar o GridSearchCV
grid_search_rating = GridSearchCV(
    estimator=xgb_rating_model,
    param_grid=param_grid_rating,
    scoring='r2', # Otimizaremos para o R²
    cv=3,
    verbose=1
)

print("\nIniciando a busca pelos melhores parâmetros (GridSearchCV)... (Isso pode levar alguns minutos)")
grid_search_rating.fit(Xr_train, yr_train)

print("\nBusca concluída.")
print(f"Melhores parâmetros encontrados: {grid_search_rating.best_params_}")

# O melhor modelo é salvo aqui
best_xgb_rating = grid_search_rating.best_estimator_


Iniciando a busca pelos melhores parâmetros (GridSearchCV)... (Isso pode levar alguns minutos)
Fitting 3 folds for each of 16 candidates, totalling 48 fits

Busca concluída.
Melhores parâmetros encontrados: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}


In [17]:
# avaliacao do modelo otimizado
print("\nAvaliando o modelo otimizado...")
pred_rating = best_xgb_rating.predict(Xr_test)

r2 = r2_score(yr_test, pred_rating)
mae = mean_absolute_error(yr_test, pred_rating)
rmse = np.sqrt(mean_squared_error(yr_test, pred_rating))

print(f"\n[Resultado IMDb Rating Otimizado] R²={r2:.3f} | MAE={mae:.3f} | RMSE={rmse:.3f}")

# salvar modelo
dump(best_xgb_rating, f"{MODELS}/modelo_nota_imdb.pkl")
print(f"\nModelo otimizado 'modelo_nota_imdb.pkl' salvo na pasta '{MODELS}' ✔️")


Avaliando o modelo otimizado...

[Resultado IMDb Rating Otimizado] R²=0.631 | MAE=0.137 | RMSE=0.177

Modelo otimizado 'modelo_nota_imdb.pkl' salvo na pasta '../models' ✔️


### Previsão Específica

In [18]:
from joblib import load
import pandas as pd
import numpy as np

# Carregar o modelo otimizado
model_path = f"{MODELS}/modelo_nota_imdb.pkl"
loaded_model = load(model_path)
print(f"Modelo '{model_path}' carregado com sucesso.")

expected_features = Xr.columns.tolist()
genre_cols_in_model = label_cols

# Dados do filme de exemplo
shawshank_data = {
    'Released_Year': 1994,
    'Runtime': 142,
    'Meta_score': 80.0,
    'No_of_Votes': 2343110,
    'Gross': 28341469.0,
    'Genre_str': 'Drama'
}

Modelo '../models/modelo_nota_imdb.pkl' carregado com sucesso.


In [19]:
# DataFrame de uma linha para a previsão
prediction_df = pd.DataFrame([shawshank_data])


# +++ a feature 'Movie_Age'
latest_year = 2025 # O ano de referência que você usou
prediction_df['Movie_Age'] = latest_year - prediction_df['Released_Year']

for genre_name in genre_cols_in_model:
    # O nome da coluna é o próprio nome do gênero
    if genre_name in prediction_df['Genre_str'].iloc[0]:
        prediction_df[genre_name] = 1
    else:
        prediction_df[genre_name] = 0

# garantir que o DataFrame de previsão tenha exatamente as mesmas colunas e na mesma ordem
prediction_df = prediction_df[expected_features]

In [20]:
# Fazer a previsão final
predicted_rating = loaded_model.predict(prediction_df)

print("--- PREVISÃO FINAL PARA O DESAFIO ---")
print(f"Filme: The Shawshank Redemption (1994)")
print(f"Nota IMDB Prevista: {predicted_rating[0]:.2f}")

--- PREVISÃO FINAL PARA O DESAFIO ---
Filme: The Shawshank Redemption (1994)
Nota IMDB Prevista: 9.06
